You have seen some examples of how to interact with JSON, CSV and make API Calls. Take some time to explore the YAHOO Finance Guide at https://financeapi.net/ which shows you endpoints for calling information. Your job is to code a Python Program that does the following things:

1. Takes user input for a stock (using the Ticker Symbol): IE with the input() command
2. You will display back the user: Ticker Name, Full Name of the Stock, Current Market Price, Target Mean Price, 52 week high and 52 week low. Also, output 5 of the current trending stock. 
3. Put that data in a DataFrame and then store that to a CSV file
4. Handle Errors (IE, the stock doesn't exist and/or the API is not returning information)

You will then have to read the documentation and figure out which modules to call to get the data that you need. You will need to make more than one call to this API. Use the examples in our Zoom Session to guide you, but the documentation will be enough. Post your code to GitHub.

In [1]:
# Import packages
import json
import pandas as pd
import requests

In [2]:
# Info for API
apikey="9GiqEC7EiZaMVBnqdu02Iaarm15T3u638OXLAbvq"

url = "https://yfapi.net/v6/finance/quote"

# Ask for input
stock = input()

# Info we want (translated into language from API)
wanted_info = [
    'symbol',
    'longName',
    'regularMarketPrice',
    'fiftyTwoWeekHigh',
    'fiftyTwoWeekLow'
]

In [14]:
# Grab everything except target mean price and trending stocks

headers = {
'x-api-key': apikey
}

response = requests.request("GET", url, headers=headers, params={"symbols":stock})
stock_json_pt1 = response.json()

df1 = pd.DataFrame(stock_json_pt1['quoteResponse']['result'])
df1 = df1[wanted_info]
df1

,symbol,longName,regularMarketPrice,fiftyTwoWeekHigh,fiftyTwoWeekLow
0,AAPL,Apple Inc.,227.725,237.23,164.08


In [43]:
# Grab target mean price

url = "https://yfapi.net/v11/finance/quoteSummary/" + stock

response = requests.request("GET", url, headers=headers, params={"symbol":stock, 'modules':'financialData'})
stock_json_pt2 = response.json()

df2 = stock_json_pt2['quoteSummary']['result'][0]['financialData']
df2 = {'targetMeanPrice': df2['targetMeanPrice']['raw']}
df2 = pd.DataFrame(df2, index = [0])
df2

,targetMeanPrice
0,240.58


In [56]:
# Grab trending stocks

url = 'https://yfapi.net/v1/finance/trending/US'

response = requests.request("GET", url, headers=headers, params={"symbol":stock, 'modules':'financialData'})
stock_json_pt3 = response.json()

In [61]:
df3 = pd.DataFrame(stock_json_pt3['finance']['result'][0]['quotes'])
df3 = df3.head(5)
df3 = df3['symbol'].tolist()
df3 = ", ".join(df3)
df3 = pd.DataFrame({'trendingStocks': df3}, index = [0])
df3

,trendingStocks
0,"^N225, NQ=F, 9866.HK, 9988.HK, ^HSI"


In [62]:
# Combine dataframes
df = pd.concat([df1, df2, df3], axis = 1)
df

,symbol,longName,regularMarketPrice,fiftyTwoWeekHigh,fiftyTwoWeekLow,targetMeanPrice,trendingStocks
0,AAPL,Apple Inc.,227.725,237.23,164.08,240.58,"^N225, NQ=F, 9866.HK, 9988.HK, ^HSI"


In [63]:
# Export to CSV
df.to_csv(stock + '_data.csv')